In [2]:
pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.9 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found exi

In [5]:
import torch
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from pathlib import Path

class GarbageDetector:
    """
    A class to load a pretrained YOLOv5 model and detect garbage in images.
    """
    
    def __init__(self, model_path, conf_threshold=0.25):
        """
        Initialize the garbage detector.
        
        Args:
            model_path (str): Path to the trained YOLOv5 model (.pt file)
            conf_threshold (float): Confidence threshold for detections (0-1)
        """
        self.conf_threshold = conf_threshold
        self.class_names = ['BIODEGRADABLE', 'CARDBOARD', 'GLASS', 'METAL', 'PAPER', 'PLASTIC']
        
        # Load the model
        print(f"Loading model from: {model_path}")
        self.model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path)
        self.model.conf = conf_threshold
        print("Model loaded successfully!")
    
    def detect_garbage(self, image_path, show_result=True, save_path=None):
        """
        Detect garbage in an image and return results.
        
        Args:
            image_path (str): Path to the input image
            show_result (bool): Whether to display the result
            save_path (str): Path to save the annotated image (optional)
            
        Returns:
            dict: Detection results containing:
                - is_garbage (bool): Whether garbage was detected
                - num_objects (int): Number of garbage objects detected
                - detections (list): List of detected objects with details
                - confidence_scores (list): Confidence scores for each detection
        """
        # Read the image
        img = cv2.imread(image_path)
        if img is None:
            raise ValueError(f"Could not read image from {image_path}")
        
        # Run inference
        results = self.model(image_path)
        
        # Parse results
        detections = results.pandas().xyxy[0]  # Predictions as pandas DataFrame
        
        num_objects = len(detections)
        is_garbage = num_objects > 0
        
        detection_list = []
        confidence_scores = []
        
        for _, row in detections.iterrows():
            detection_info = {
                'class': self.class_names[int(row['class'])],
                'confidence': float(row['confidence']),
                'bbox': [int(row['xmin']), int(row['ymin']), 
                        int(row['xmax']), int(row['ymax'])]
            }
            detection_list.append(detection_info)
            confidence_scores.append(float(row['confidence']))
        
        # Create result dictionary
        result = {
            'is_garbage': is_garbage,
            'num_objects': num_objects,
            'detections': detection_list,
            'confidence_scores': confidence_scores,
            'image_path': image_path
        }
        
        # Visualize if requested
        if show_result or save_path:
            annotated_img = self._draw_detections(img.copy(), detection_list)
            
            if show_result:
                self._display_result(annotated_img, result)
            
            if save_path:
                cv2.imwrite(save_path, annotated_img)
                print(f"Annotated image saved to: {save_path}")
        
        return result
    
    def _draw_detections(self, img, detections):
        """Draw bounding boxes and labels on the image."""
        for det in detections:
            x1, y1, x2, y2 = det['bbox']
            label = det['class']
            conf = det['confidence']
            
            # Draw rectangle
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            
            # Draw label with confidence
            text = f"{label}: {conf:.2f}"
            (text_width, text_height), _ = cv2.getTextSize(
                text, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2
            )
            cv2.rectangle(img, (x1, y1 - text_height - 10), 
                         (x1 + text_width, y1), (0, 255, 0), -1)
            cv2.putText(img, text, (x1, y1 - 5), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)
        
        return img
    
    def _display_result(self, img, result):
        """Display the detection result with matplotlib."""
        # Convert BGR to RGB
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        plt.figure(figsize=(12, 8))
        plt.imshow(img_rgb)
        plt.axis('off')
        
        # Create title with detection info
        if result['is_garbage']:
            title = f"✓ GARBAGE DETECTED - {result['num_objects']} object(s) found"
            title_color = 'red'
        else:
            title = "✗ NO GARBAGE DETECTED"
            title_color = 'green'
        
        plt.title(title, fontsize=14, color=title_color, weight='bold')
        plt.tight_layout()
        plt.show()
    
    def print_summary(self, result):
        """Print a detailed summary of detection results."""
        print("\n" + "="*60)
        print("GARBAGE DETECTION SUMMARY")
        print("="*60)
        print(f"Image: {result['image_path']}")
        print(f"Garbage Detected: {'YES' if result['is_garbage'] else 'NO'}")
        print(f"Number of Objects: {result['num_objects']}")
        
        if result['num_objects'] > 0:
            print("\nDetected Objects:")
            print("-" * 60)
            for i, det in enumerate(result['detections'], 1):
                print(f"{i}. {det['class']} (Confidence: {det['confidence']:.2%})")
        
        print("="*60 + "\n")


# ============================================================================
# USAGE EXAMPLE
# ============================================================================

if __name__ == "__main__":
    # Initialize the detector
    # Replace with your actual model path
    MODEL_PATH = "/kaggle/input/uusha-data-4/best.pt"
    
    detector = GarbageDetector(
        model_path=MODEL_PATH,
        conf_threshold=0.25
    )
    
    # Example 1: Detect garbage in a single image
    image_path = "/kaggle/input/garbage-detection-yolov5/data/images/train/biodegradable1245_jpg.rf.85d17c8e459fd785141d715d8d118d2e.jpg"
    
    result = detector.detect_garbage(
        image_path=image_path,
        show_result=True,
        save_path="output_detection.jpg"
    )
    
    # Print detailed summary
    detector.print_summary(result)
    
    # Example 2: Process multiple images
    test_images = [
        "/kaggle/input/garbage-detection-yolov5/data/images/train/biodegradable1237_jpg.rf.555f96874ef6fdaa8cd5cd74ea10dca1.jpg",
        "/kaggle/input/garbage-detection-yolov5/data/images/train/biodegradable1240_jpg.rf.82612fec6d82879618c1ee66abac5d44.jpg",
        "/kaggle/input/garbage-detection-yolov5/data/images/train/biodegradable1229_jpg.rf.3fd65b69cedb51a5d0e9ce25a26edddf.jpg"
    ]
    
    for img_path in test_images:
        result = detector.detect_garbage(img_path, show_result=False)
        
        # Simple yes/no output
        status = "GARBAGE FOUND" if result['is_garbage'] else "NO GARBAGE"
        print(f"{img_path}: {status} ({result['num_objects']} objects)")
    
    # Example 3: Quick check - just yes/no
    def is_image_garbage(image_path):
        """Simple function to check if image contains garbage."""
        result = detector.detect_garbage(image_path, show_result=False)
        return result['is_garbage']
    
    # Use it
    if is_image_garbage("/kaggle/input/garbage-detection-yolov5/data/images/train/biodegradable1237_jpg.rf.555f96874ef6fdaa8cd5cd74ea10dca1.jpg"):
        print("This image contains garbage!")
    else:
        print("This image is clean!")

Loading model from: /kaggle/input/uusha-data-4/best.pt
requirements: Ultralytics requirement ['urllib3>=2.5.0 ; python_version > "3.8"'] not found, attempting AutoUpdate...
WARNING ⚠️ Retry 1/2 failed: Command 'uv pip install --no-cache-dir "urllib3>=2.5.0 ; python_version > "3.8""  --index-strategy=unsafe-best-match --break-system-packages --prerelease=allow' returned non-zero exit status 2.


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master


WARNING ⚠️ Retry 2/2 failed: Command 'uv pip install --no-cache-dir "urllib3>=2.5.0 ; python_version > "3.8""  --index-strategy=unsafe-best-match --break-system-packages --prerelease=allow' returned non-zero exit status 2.
WARNING ⚠️ requirements: ❌ Command 'uv pip install --no-cache-dir "urllib3>=2.5.0 ; python_version > "3.8""  --index-strategy=unsafe-best-match --break-system-packages --prerelease=allow' returned non-zero exit status 2.


YOLOv5 🚀 2025-11-3 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)

Fusing layers... 
Model summary: 157 layers, 7026307 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


Model loaded successfully!


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Annotated image saved to: output_detection.jpg

GARBAGE DETECTION SUMMARY
Image: /kaggle/input/garbage-detection-yolov5/data/images/train/biodegradable1245_jpg.rf.85d17c8e459fd785141d715d8d118d2e.jpg
Garbage Detected: YES
Number of Objects: 62

Detected Objects:
------------------------------------------------------------
1. BIODEGRADABLE (Confidence: 90.08%)
2. BIODEGRADABLE (Confidence: 89.80%)
3. BIODEGRADABLE (Confidence: 89.76%)
4. BIODEGRADABLE (Confidence: 89.20%)
5. BIODEGRADABLE (Confidence: 89.04%)
6. BIODEGRADABLE (Confidence: 88.88%)
7. BIODEGRADABLE (Confidence: 88.80%)
8. BIODEGRADABLE (Confidence: 88.57%)
9. BIODEGRADABLE (Confidence: 88.51%)
10. BIODEGRADABLE (Confidence: 88.30%)
11. BIODEGRADABLE (Confidence: 88.12%)
12. BIODEGRADABLE (Confidence: 88.03%)
13. BIODEGRADABLE (Confidence: 87.87%)
14. BIODEGRADABLE (Confidence: 87.75%)
15. BIODEGRADABLE (Confidence: 87.70%)
16. BIODEGRADABLE (Confidence: 87.60%)
17. BIODEGRADABLE (Confidence: 87.50%)
18. BIODEGRADABLE (Con

/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:898: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


In [ ]:
# /kaggle/input/garbage-detection-yolov5/data/images/train/biodegradable1245_jpg.rf.85d17c8e459fd785141d715d8d118d2e.jpg